In [1]:
from keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
from keras import optimizers, regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten, GRU
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils, to_categorical
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
import pickle
import re
import operator
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
corpus = pickle.load(open('corpus233.pkl', 'rb'))
cleaned = [re.sub('[^a-zA-Z ]', '', song) for song in corpus]

cleaned = list(set(cleaned))
tokens = [song.split() for song in cleaned]

words = []
for song in cleaned:
    for word in song.split():
        words.append(word)

lines = []
for i in range(len(words)-7):
    lines.append(words[i:i+7])

In [3]:
lines_unique = []
for line in tqdm(lines):
    if line not in lines_unique:
        lines_unique.append(line)

len(lines_unique)

100%|██████████████████████████████████████████████████████████████████████████| 57734/57734 [00:38<00:00, 1491.95it/s]


40857

In [4]:
pickle.dump(lines_unique, open('unique_lines.pkl', 'wb'))

In [5]:
lines_unique = pickle.load(open('unique_lines.pkl', 'rb'))

In [6]:
#Turn words to ints
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
vocab_size = len(tokenizer.word_index) + 1
pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))
sequences = np.array(sequences)

In [7]:
sequences.shape

(57734, 7)

In [8]:
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [9]:
#Create model *DONT USE DROPOUT - DEPENDENCY ERROR*
model = Sequential()
model.add(Embedding(vocab_size, output_dim=50, input_length=seq_length))
model.add(GRU(256, return_sequences=True))
model.add(GRU(256))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
sgd = optimizers.SGD(lr=0.1, decay=0, momentum=0.05, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6, 50)             390650    
_________________________________________________________________
gru_1 (GRU)                  (None, 6, 256)            235776    
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               393984    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               25700     
_________________________________________________________________
dense_2 (Dense)              (None, 7813)              789113    
Total params: 1,835,223
Trainable params: 1,835,223
Non-trainable params: 0
_________________________________________________________________


In [12]:
filepath = "models/weights.{epoch:02d}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=False, mode="min", period = 5)
callbacks_list = [checkpoint]

model.fit(X, y, batch_size=256, epochs=150, callbacks=callbacks_list, verbose=2)

Epoch 1/150


55040/57734 [===========================>..] - ETA: 32s - loss: 8.9413 - acc: 0.03 - ETA: 31s - loss: 8.9403 - acc: 0.02 - ETA: 31s - loss: 8.9388 - acc: 0.03 - ETA: 30s - loss: 8.9376 - acc: 0.04 - ETA: 30s - loss: 8.9373 - acc: 0.04 - ETA: 30s - loss: 8.9370 - acc: 0.04 - ETA: 30s - loss: 8.9356 - acc: 0.04 - ETA: 30s - loss: 8.9359 - acc: 0.04 - ETA: 30s - loss: 8.9357 - acc: 0.04 - ETA: 30s - loss: 8.9353 - acc: 0.04 - ETA: 30s - loss: 8.9349 - acc: 0.04 - ETA: 30s - loss: 8.9341 - acc: 0.04 - ETA: 30s - loss: 8.9332 - acc: 0.04 - ETA: 29s - loss: 8.9327 - acc: 0.04 - ETA: 29s - loss: 8.9323 - acc: 0.04 - ETA: 29s - loss: 8.9320 - acc: 0.04 - ETA: 29s - loss: 8.9321 - acc: 0.04 - ETA: 29s - loss: 8.9316 - acc: 0.04 - ETA: 29s - loss: 8.9310 - acc: 0.04 - ETA: 29s - loss: 8.9307 - acc: 0.04 - ETA: 29s - loss: 8.9305 - acc: 0.04 - ETA: 29s - loss: 8.9302 - acc: 0.04 - ETA: 29s - loss: 8.9301 - acc: 0.03 - ETA: 29s - loss: 8.9298 - acc: 0.03 - ETA: 29s - loss: 8.9297 - acc: 0.03 - ETA

57734/57734 [==============================] - ETA: 1s - loss: 8.3576 - acc: 0.035 - ETA: 1s - loss: 8.3516 - acc: 0.035 - ETA: 1s - loss: 8.3438 - acc: 0.035 - ETA: 1s - loss: 8.3366 - acc: 0.035 - ETA: 0s - loss: 8.3293 - acc: 0.035 - ETA: 0s - loss: 8.3238 - acc: 0.035 - ETA: 0s - loss: 8.3185 - acc: 0.035 - ETA: 0s - loss: 8.3111 - acc: 0.035 - ETA: 0s - loss: 8.3042 - acc: 0.035 - ETA: 0s - loss: 8.2976 - acc: 0.035 - 40s 694us/step - loss: 8.2934 - acc: 0.0356
Epoch 2/150


55040/57734 [===========================>..] - ETA: 34s - loss: 6.7451 - acc: 0.02 - ETA: 34s - loss: 6.7290 - acc: 0.02 - ETA: 35s - loss: 6.7996 - acc: 0.02 - ETA: 35s - loss: 6.7821 - acc: 0.02 - ETA: 35s - loss: 6.7554 - acc: 0.03 - ETA: 34s - loss: 6.7633 - acc: 0.03 - ETA: 34s - loss: 6.7407 - acc: 0.03 - ETA: 34s - loss: 6.7651 - acc: 0.03 - ETA: 33s - loss: 6.7727 - acc: 0.03 - ETA: 33s - loss: 6.7681 - acc: 0.03 - ETA: 33s - loss: 6.7660 - acc: 0.03 - ETA: 32s - loss: 6.7811 - acc: 0.03 - ETA: 32s - loss: 6.7764 - acc: 0.03 - ETA: 32s - loss: 6.7640 - acc: 0.03 - ETA: 32s - loss: 6.7579 - acc: 0.03 - ETA: 31s - loss: 6.7837 - acc: 0.03 - ETA: 31s - loss: 6.7672 - acc: 0.03 - ETA: 31s - loss: 6.7591 - acc: 0.03 - ETA: 31s - loss: 6.7678 - acc: 0.03 - ETA: 31s - loss: 6.7680 - acc: 0.03 - ETA: 31s - loss: 6.7671 - acc: 0.03 - ETA: 30s - loss: 6.7613 - acc: 0.03 - ETA: 30s - loss: 6.7609 - acc: 0.03 - ETA: 30s - loss: 6.7562 - acc: 0.03 - ETA: 30s - loss: 6.7557 - acc: 0.03 - ETA

57734/57734 [==============================] - ETA: 1s - loss: 6.7653 - acc: 0.033 - ETA: 1s - loss: 6.7659 - acc: 0.033 - ETA: 1s - loss: 6.7663 - acc: 0.033 - ETA: 1s - loss: 6.7668 - acc: 0.033 - ETA: 0s - loss: 6.7662 - acc: 0.033 - ETA: 0s - loss: 6.7673 - acc: 0.033 - ETA: 0s - loss: 6.7670 - acc: 0.033 - ETA: 0s - loss: 6.7661 - acc: 0.033 - ETA: 0s - loss: 6.7671 - acc: 0.033 - ETA: 0s - loss: 6.7684 - acc: 0.033 - 37s 638us/step - loss: 6.7683 - acc: 0.0336
Epoch 3/150


...

57734/57734 [==============================] - ETA: 1s - loss: 1.6688 - acc: 0.624 - ETA: 1s - loss: 1.6689 - acc: 0.624 - ETA: 1s - loss: 1.6691 - acc: 0.623 - ETA: 1s - loss: 1.6685 - acc: 0.623 - ETA: 1s - loss: 1.6688 - acc: 0.623 - ETA: 0s - loss: 1.6690 - acc: 0.623 - ETA: 0s - loss: 1.6697 - acc: 0.623 - ETA: 0s - loss: 1.6708 - acc: 0.623 - ETA: 0s - loss: 1.6705 - acc: 0.623 - ETA: 0s - loss: 1.6713 - acc: 0.623 - 41s 708us/step - loss: 1.6720 - acc: 0.6231
Epoch 149/150


55040/57734 [===========================>..] - ETA: 32s - loss: 1.7564 - acc: 0.60 - ETA: 34s - loss: 1.5095 - acc: 0.66 - ETA: 35s - loss: 1.4665 - acc: 0.67 - ETA: 35s - loss: 1.4565 - acc: 0.67 - ETA: 35s - loss: 1.4519 - acc: 0.67 - ETA: 35s - loss: 1.4551 - acc: 0.67 - ETA: 35s - loss: 1.4449 - acc: 0.67 - ETA: 35s - loss: 1.4341 - acc: 0.68 - ETA: 35s - loss: 1.4242 - acc: 0.68 - ETA: 35s - loss: 1.4173 - acc: 0.68 - ETA: 35s - loss: 1.4174 - acc: 0.67 - ETA: 34s - loss: 1.4065 - acc: 0.68 - ETA: 34s - loss: 1.4256 - acc: 0.67 - ETA: 34s - loss: 1.4337 - acc: 0.67 - ETA: 34s - loss: 1.4370 - acc: 0.67 - ETA: 34s - loss: 1.4420 - acc: 0.67 - ETA: 33s - loss: 1.4588 - acc: 0.67 - ETA: 33s - loss: 1.4581 - acc: 0.67 - ETA: 33s - loss: 1.4554 - acc: 0.67 - ETA: 32s - loss: 1.4611 - acc: 0.67 - ETA: 32s - loss: 1.4543 - acc: 0.67 - ETA: 34s - loss: 1.4600 - acc: 0.67 - ETA: 38s - loss: 1.4658 - acc: 0.67 - ETA: 41s - loss: 1.4698 - acc: 0.67 - ETA: 44s - loss: 1.4741 - acc: 0.67 - ETA

57734/57734 [==============================] - ETA: 1s - loss: 1.6059 - acc: 0.640 - ETA: 1s - loss: 1.6068 - acc: 0.640 - ETA: 1s - loss: 1.6070 - acc: 0.640 - ETA: 1s - loss: 1.6077 - acc: 0.640 - ETA: 1s - loss: 1.6093 - acc: 0.639 - ETA: 0s - loss: 1.6088 - acc: 0.639 - ETA: 0s - loss: 1.6097 - acc: 0.639 - ETA: 0s - loss: 1.6109 - acc: 0.639 - ETA: 0s - loss: 1.6113 - acc: 0.639 - ETA: 0s - loss: 1.6126 - acc: 0.638 - 44s 756us/step - loss: 1.6134 - acc: 0.6388
Epoch 150/150


55040/57734 [===========================>..] - ETA: 33s - loss: 2.2530 - acc: 0.50 - ETA: 33s - loss: 2.0987 - acc: 0.52 - ETA: 34s - loss: 2.0052 - acc: 0.55 - ETA: 35s - loss: 1.9193 - acc: 0.57 - ETA: 35s - loss: 1.8609 - acc: 0.58 - ETA: 35s - loss: 1.8058 - acc: 0.60 - ETA: 35s - loss: 1.7773 - acc: 0.60 - ETA: 35s - loss: 1.7191 - acc: 0.62 - ETA: 36s - loss: 1.6853 - acc: 0.63 - ETA: 36s - loss: 1.6460 - acc: 0.64 - ETA: 36s - loss: 1.6344 - acc: 0.63 - ETA: 36s - loss: 1.6130 - acc: 0.64 - ETA: 36s - loss: 1.6043 - acc: 0.64 - ETA: 35s - loss: 1.6021 - acc: 0.64 - ETA: 35s - loss: 1.5799 - acc: 0.64 - ETA: 35s - loss: 1.5705 - acc: 0.64 - ETA: 35s - loss: 1.5583 - acc: 0.65 - ETA: 35s - loss: 1.5442 - acc: 0.65 - ETA: 34s - loss: 1.5351 - acc: 0.65 - ETA: 34s - loss: 1.5351 - acc: 0.65 - ETA: 34s - loss: 1.5282 - acc: 0.65 - ETA: 33s - loss: 1.5165 - acc: 0.66 - ETA: 33s - loss: 1.5172 - acc: 0.66 - ETA: 33s - loss: 1.5123 - acc: 0.66 - ETA: 33s - loss: 1.5073 - acc: 0.66 - ETA

57734/57734 [==============================] - ETA: 1s - loss: 1.5858 - acc: 0.643 - ETA: 1s - loss: 1.5872 - acc: 0.643 - ETA: 1s - loss: 1.5875 - acc: 0.643 - ETA: 1s - loss: 1.5877 - acc: 0.643 - ETA: 0s - loss: 1.5874 - acc: 0.643 - ETA: 0s - loss: 1.5879 - acc: 0.643 - ETA: 0s - loss: 1.5887 - acc: 0.643 - ETA: 0s - loss: 1.5888 - acc: 0.643 - ETA: 0s - loss: 1.5889 - acc: 0.643 - ETA: 0s - loss: 1.5888 - acc: 0.643 - 38s 658us/step - loss: 1.5889 - acc: 0.6432

Epoch 00150: saving model to models/weights.150-bigger.hdf5


NameError: name 'lyrics' is not defined

In [15]:
from bs4 import BeautifulSoup

ImportError: cannot import name 'HTMLParseError' from 'html.parser' (C:\Users\Cairo\Anaconda3\lib\html\parser.py)